# Implementation of the EBfilter by genomon

* EBrun (originally EBFilter) is an argparse wrapper passing command line arguments to run.py (is not needed for internal use)
* passed arguments:
    * targetMutationFile: the .vcf or .anno containing the mutations – needed --> mut_file
    * targetBamPath: path to the tumor bam file (+.bai) – needed --> tumor_bam
    * controlBamPathList: text list of path to PoN bam files (+ .bai) – needed --> pon_list
    * outputPath: clear  – needed --> output_path
    * -f option for anno or vcf – not needed --> will be inferred from .ext
    * thread_num: –not needed --> taken from config
    * -q option for quality threshold – not needed --> default _q config
    * -Q option for base quality threshold - not needed --> default _Q from config
    * --ff option for filter flags – not needed because of preprocessing??
    * --loption for samtools mpileup -l option – must elaborate..
    * --region option for restriction of regions on mpileup -l – must elaborate..
    * --debug – not needed

## Initiation

### imports

In [1]:
import os
from code import run

### snakemake config

In [2]:
config = {'EB':{'run': True}}
params = {}
params['map_quality'] = 20
params['base_quality'] = 15
params['filter_flags'] = 'UNMAP,SECONDARY,QCFAIL,DUP'
params['loption'] = True
config['EB']['threads'] = 1
config['EB']['params'] = params

### load the config and GLOBAL STATE into variable _

In [3]:
debug_mode = True
args = {}
params = config['EB']['params']
threads = config['EB']['threads']
_q = str(params['map_quality'])  # mapping quality=20
_Q = params['base_quality']      # base quality=15
filter_quals = ''
for qual in range( 33, 33 + _Q ): 
    filter_quals += chr( qual )  # qual asciis for filtering out
_ff = params['filter_flags']     # 'UNMAP,SECONDARY,QCFAIL,DUP'
is_loption = params['loption']   # True
log_path = 'output/logs'         # log_file: 'output/logs/'


### function args
will be passed during the function call

In [4]:
args['mut_file'] = 'test/anno/test_rel.csv'
args['tumor_bam'] = 'test/bam/test_Rel1.bam'
args['pon_list'] = 'test/aml_pon.list'
args['output_path'] = 'output/test_rel_eb.csv'
args['region'] = ''
log_file = os.path.join(log_path, f"{os.path.splitext(os.path.basename(args['output_path']))[0]}.log")
# dump into _
_ = {'q':_q, 'Q':_Q, 'filter_quals': filter_quals, 'ff':_ff, 'log':log_file, 'threads':threads, 'debug_mode':debug_mode}

In [5]:
filter_quals

'!"#$%&\'()*+,-./'

## Single threading

In [6]:
_['threads'] = 1
_['debug_mode'] = False
run.main(args,_)
!ls output

A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2 1 2 2 1 3 2]
A [0 3 2 1 2

T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2 3 1 2 3 2 1]
T [0 2 4 4 2

T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
T [0 4 4 4 4 7 3 7 3 1 3]
G [ 0 12 16 11 22 16 22 12 17 10 16]
G [ 0 12 16 11 22 16 22 12 17 10 16]
G [ 0 12 16 11 22 16 22 12 17 10 16]
G [ 0 12 16 11 22 16 22 12 17 10 16]
G [ 0 12 16 11 22 16 22 12 17 10 16]
G [ 0 12 16 11 22 16 22 12 17 10 16]
G [ 0 12 16 11 22 16 22 12 17 10 16]
G [ 0 12 16 1

C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C [ 0 22 26 17 40 31 27 20 26 14 24]
C

G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G [ 0 29 25 26 40 36 29 23 30 17 24]
G

...

A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A [ 0  6 26  6 19 14 18 24 12 17 16]
A

C [ 0  6 18  6 13 14 14 20  9 12 12]
C [ 0  6 18  6 13 14 14 20  9 12 12]
C [ 0  6 18  6 13 14 14 20  9 12 12]
C [ 0  6 18  6 13 14 14 20  9 12 12]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C [ 0  5 12 13 20 12 12 12  7  5 13]
C

G [0 2 5 4 3 3 1 3 1 5 1]
G [0 2 5 4 3 3 1 3 1 5 1]
G [0 2 5 4 3 3 1 3 1 5 1]
G [0 2 5 4 3 3 1 3 1 5 1]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2 1 2 2 3 2 3]
G [0 2 3 2 2

-T [0 1 2 2 1 1 1 1 2 1 1]
-T [0 1 2 2 1 1 1 1 2 1 1]
-T [0 1 2 2 1 1 1 1 2 1 1]
-T [0 1 2 2 1 1 1 1 2 1 1]
-T [0 2 1 4 0 0 0 1 1 0] [0 3 2 4 0 0 0 2 1 0]
-T [0 1 0 0 1 1 1 1 0 1 1]


ValueError: math domain error

In [13]:
# math breaks at:
# targets: 1 0 5 3
# controls: [1, 1, 2, 1, 3, 1, 2, 0, 1, 4] [0, 3, 2, 4, 0, 0, 0, 2, 1, 0] [1, 2, 3, 1, 3, 1, 2, 1, 1, 4] [0, 2, 1, 4, 0, 0, 0, 1, 1, 0]
import math
import numpy as np
import re
from functools import reduce
alpha = 20
beta = 20
Ns = np.array([0,2, 1, 4, 0, 0, 0, 1, 1, 0])
Ks = np.array([0, 3, 2, 4, 0, 0, 0, 2, 1, 0])
what = np.r_[0, Ns - Ks + 1]
reduce(lambda a, b: a + math.lgamma(b), np.r_[0, Ns - Ks + 1])


ValueError: math domain error

* it all breaks at math.lgamma(0)
* Ns - Ks +1 becomes 0 when Ns +1 = Ks
+ NS = [0,2, 1, 4, 0, 0, 0, 1, 1, 0] --> depth_control
+ Ks = [0, 3, 2, 4, 0, 0, 0, 2, 1, 0] --> vars_control
* at var = -T this is the case:
* controls = [1, 1, 2, 1, 3, 1, 2, 0, 1, 4] [0, 3, 2, 4, 0, 0, 0, 2, 1, 0] [1, 2, 3, 1, 3, 1, 2, 1, 1, 4] [0, 2, 1, 4, 0, 0, 0, 1, 1, 0]+
* call: fit_beta_binomial(np.array(depth_control_n), np.array(vars_control_n), var)
  --> depth

In [19]:
math.lgamma(1)

0.0

### indel detection in var_count_check

hrello


In [18]:
import re
indel_re = re.compile(r'([\+\-])([0-9]+)([ACGTNacgtn]+)')
read = "C+12TGCCCTACTGCAC+12TGCCCTACTGCAC+12TGCCCTACTGCAC+12TGCCCTACTGCAc+12tgccctactgcac+12tgccctactgcaC+12TGCCCTACTGCA"
var = 'TGCCCTACTGCA'
indel=False
while indel_re.search(read):
    m = indel_re.search(read)
    site = m.start()
    type = m.group(1)
    indel_size = int(m.group(2))
    varChar = m.group(3)[0:indel_size+1]
    read = read[:m.start()] + read[m.end()-1:]
    indel=True
    print(read)
if indel:
    read = read[:-1]
print(read)

CC+12TGCCCTACTGCAC+12TGCCCTACTGCAC+12TGCCCTACTGCAc+12tgccctactgcac+12tgccctactgcaC+12TGCCCTACTGCA
CCC+12TGCCCTACTGCAC+12TGCCCTACTGCAc+12tgccctactgcac+12tgccctactgcaC+12TGCCCTACTGCA
CCCC+12TGCCCTACTGCAc+12tgccctactgcac+12tgccctactgcaC+12TGCCCTACTGCA
CCCCc+12tgccctactgcac+12tgccctactgcaC+12TGCCCTACTGCA
CCCCcc+12tgccctactgcaC+12TGCCCTACTGCA
CCCCccC+12TGCCCTACTGCA
CCCCccCA
CCCCccC


## Multithreading

In [ ]:
_['threads'] = 3
_['debug_mode'] = True
run.main(args,_)
!ls output/

In [ ]:
ls output/

In [ ]:
import pandas as pd
# create short sample file from test_Rel1-CR1_bull.csv
test_rel = pd.read_csv('test/anno/test_Rel1-CR1_bull.csv')
test_rel.sort_values(['Chr', 'Start'], ascending=True, inplace=True)
test_rel.iloc[::3,:7].to_csv('test/anno/test_rel.csv', sep='\t', index=None)
test_rel

## using advanced dataframes for getting the pileup files 

### get anno and pileup as dataframes (too big?)

In [ ]:
anno = pd.read_csv('testdata/input.anno', sep='\t', header=None, names=['Chr','Start', 'End', 'ref', 'var'], dtype={'Chr': str, 'Start':int, 'End': int})
tumorpileup = pd.read_csv('output/tumor.pileup', sep='\t', header=None, names=['Chr', 'Start', 'ref', 'depth', 'reads', 'mapQ'], dtype={'Start':int, 'reads':str, 'mapQ': str})
pd.merge(anno,tumorpileup, on=['Chr', 'Start'])


In [ ]:
threads = 4
split = round(len(tumorpileup.index) / threads)
splits = np.array_split(tumorpileup, 4)

In [ ]:
import pandas as pd
anno = pd.read_csv('testdata/input.anno', sep='\t', header=None)
anno

In [ ]:
anno[1] = anno[1] - 1 - (anno[3] == '-')
anno[2] = anno[2] - (anno[3] == '-')
anno

In [ ]:
anno